In [2]:
import pandas as pd
import numpy as np
for module in pd, np:
    print(module.__name__, module.__version__)

pandas 1.1.5
numpy 1.18.5


## 一. cat对象的属性
1. 在 pandas 中提供了 category 类型，使用户能够处理分类类型的变量，将一个普通序列转换成分类变量可以使用 astype 方法。

In [3]:
df = pd.read_csv('learn_pandas.csv',
      usecols = ['Grade', 'Name', 'Gender', 'Height', 'Weight'])
df.head(3)

,Grade,Name,Gender,Height,Weight
0,Freshman,Gaopeng Yang,Female,158.9,46.0
1,Freshman,Changqiang You,Male,166.5,70.0
2,Senior,Mei Sun,Male,188.9,89.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Grade   200 non-null    object 
 1   Name    200 non-null    object 
 2   Gender  200 non-null    object 
 3   Height  183 non-null    float64
 4   Weight  189 non-null    float64
dtypes: float64(2), object(3)
memory usage: 7.9+ KB


In [5]:
# Grade, Gender,因为包含字符串，默认为 为object类型，可以转换成cat类型
for col in ['Grade', 'Gender']:
    df[col] = df[col].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   Grade   200 non-null    category
 1   Name    200 non-null    object  
 2   Gender  200 non-null    category
 3   Height  183 non-null    float64 
 4   Weight  189 non-null    float64 
dtypes: category(2), float64(2), object(1)
memory usage: 5.5+ KB


 - 在一个分类类型的 Series 中定义了 cat 对象，它和上一章中介绍的 str 对象类似，定义了一些属性和方法来进行分类类别的操作。
 - 对于一个具体的分类，有两个组成部分，其一为类别的本身，它以Index类型存储，其二为是否有序，它们都可以通过cat的属性被访问

In [6]:
s = df.Grade
print(type(s)) 
s.cat

<class 'pandas.core.series.Series'>


In [7]:
print(s.cat.categories)
print(s.cat.ordered)

Index(['Freshman', 'Junior', 'Senior', 'Sophomore'], dtype='object')
False


### 2. 类别的增加、删除和修改
通过 cat 对象的 categories 属性能够完成对类别的查询，那么应该如何进行“增改查删”的其他三个操作呢？

### 注意：类别不得直接修改
#### 在第三章中曾提到，索引 Index 类型是无法用 index_obj[0] = item 来修改的，
#### 而 categories 被存储在 Index 中，因此 pandas 在 cat 属性上定义了若干方法来达到相同的目的

In [8]:
# 增加： add_categories:
s = s.cat.add_categories('General')
s.cat.categories

Index(['Freshman', 'Junior', 'Senior', 'Sophomore', 'General'], dtype='object')

In [9]:
# 删除： remove_categories,同时所有原来序列中的该类会被设置为缺失。
s = s.cat.remove_categories('Freshman')
s.cat.categories

Index(['Junior', 'Senior', 'Sophomore', 'General'], dtype='object')

In [10]:
s.head()

0          NaN
1          NaN
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Junior', 'Senior', 'Sophomore', 'General']

此外可以使用 set_categories 直接设置序列的新类别，原来的类别中如果存在元素不属于新类别，那么会被设置为缺失。

In [11]:
s = s.cat.set_categories(['cainiao', 'laosiji', 'Senior'])
s.cat.categories

Index(['cainiao', 'laosiji', 'Senior'], dtype='object')

In [12]:
s.head()

0       NaN
1       NaN
2    Senior
3       NaN
4       NaN
Name: Grade, dtype: category
Categories (3, object): ['cainiao', 'laosiji', 'Senior']

“增改查删”中还剩下修改的操作，这可以通过 rename_categories 方法完成，
同时需要注意的是，这个方法会对原序列的对应值也进行相应修改。
例如，现在把 Sophomore 改成中文的 本科二年级学生 ：

In [13]:
s = df['Grade']
print(s.cat.categories)
print(s.head())
print('-' * 10)

s = s.cat.rename_categories({'Freshman' : 'Cainiao', 'Sophomore' : 'laosiji'})
print(s.cat.categories)
s.head()

Index(['Freshman', 'Junior', 'Senior', 'Sophomore'], dtype='object')
0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Junior', 'Senior', 'Sophomore']
----------
Index(['Cainiao', 'Junior', 'Senior', 'laosiji'], dtype='object')


0    Cainiao
1    Cainiao
2     Senior
3    laosiji
4    laosiji
Name: Grade, dtype: category
Categories (4, object): ['Cainiao', 'Junior', 'Senior', 'laosiji']

## 二、有序分类
1. 序的建立
 - 有序类别和无序类别可以通过 as_unordered 和 reorder_categories 互相转化
 - 需要注意的是后者传入的参数必须是由当前序列的无需类别构成的列表，不能够增加新的类别，也不能缺少原来的类别，并且必须指定参数 ordered=True ，否则方法无效。
 - 下面举例：
   - 对年级高低进行相对大小的类别划分，然后再恢复无序状态：

In [14]:
s = df['Grade'].astype('category')
s = s.cat.reorder_categories(['Freshman', 'Sophomore', 'Junior', 'Senior'],ordered=True)
s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman' < 'Sophomore' < 'Junior' < 'Senior']

In [15]:
s.cat.as_unordered().head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Sophomore', 'Junior', 'Senior']

2. 排序和比较
对于分类变量的排序：
只需把列的类型修改为category后，再赋予相应的大小关系，就能正常地使用sort_index和sort_values。例如，对年级进行排序：

In [17]:
df.Grade = df.Grade.astype('category')
df.Grade = df.Grade.cat.reorder_categories(['Freshman', 'Sophomore', 'Junior', 'Senior'],ordered=True)
df.sort_values(by = 'Grade', ascending = False).head(10) # 值排序, 降序排列

,Grade,Name,Gender,Height,Weight
100,Senior,Xiaofeng Shi,Female,164.4,55.0
87,Senior,Feng Yang,Female,167.0,52.0
79,Senior,Changmei Sun,Female,155.3,46.0
78,Senior,Li Xu,Female,161.5,53.0
77,Senior,Gaopeng Qin,Female,159.4,52.0
156,Senior,Juan Qin,Female,156.0,47.0
161,Senior,Quan Qian,Female,159.0,50.0
66,Senior,Chengpeng Zhou,Male,177.1,81.0
165,Senior,Feng Han,Male,170.1,69.0
123,Senior,Qiang Shi,Female,157.7,NaN


In [18]:
df.set_index('Grade').sort_index(ascending = False).head() # 索引排序

,Name,Gender,Height,Weight
Grade,,,,
Senior,Xiaofeng Shi,Female,164.4,55.0
Senior,Feng Yang,Female,167.0,52.0
Senior,Changmei Sun,Female,155.3,46.0
Senior,Li Xu,Female,161.5,53.0
Senior,Gaopeng Qin,Female,159.4,52.0


### 由于序的建立，因此就可以进行比较操作。分类变量的比较操作分为两类
1. 第一种： ==或!=关系的比较，比较的对象可以是标量或者同长度的Series（或list）
2. 第二种：>,>=,<,<=四类大小关系的比较，比较的对象和第一种类似，但是所有参与比较的元素必须属于原序列的categories，同时要和原序列具有相同的索引

In [21]:
res1 = df['Grade'] == 'Freshman'
res1.head(10)

0     True
1     True
2    False
3    False
4    False
5     True
6     True
7    False
8     True
9    False
Name: Grade, dtype: bool

In [22]:
res2 = df.Grade <= 'Sophomore'
res2.head(10)

0     True
1     True
2    False
3     True
4     True
5     True
6     True
7    False
8     True
9    False
Name: Grade, dtype: bool

## 三、区间类别
1. 利用cut和qcut进行区间构造
区间是一种特殊的类别，在实际数据分析中，区间序列往往是通过cut和qcut方法进行构造的，这两个函数能够把原序列的数值特征进行装箱，即用区间位置来代替原来的具体数值。

首先介绍cut的常见用法：

其中，最重要的参数是bin，如果传入整数n，则代表把整个传入数组的按照最大和最小值等间距地分为n段。由于区间默认是左开右闭，需要进行调整把最小值包含进去，在pandas中的解决方案是在值最小的区间左端点再减去0.001*(max-min)，因此如果对序列[1,2]划分为2个箱子时，第一个箱子的范围(0.999,1.5]，第二个箱子的范围是(1.5,2]。如果需要指定左闭右开时，需要把right参数设置为False，相应的区间调整方法是在值最大的区间右端点再加上0.001*(max-min)。

In [24]:
s = df['Height']
print(s.max(), s.min(), s.mean())
s.head(10)

193.9 145.4 163.21803278688526


0    158.9
1    166.5
2    188.9
3      NaN
4    174.0
5    158.0
6    162.5
7    161.9
8    163.0
9    164.8
Name: Height, dtype: float64

In [26]:
res = pd.cut(s, bins = 5)
res.head(10)

0    (155.1, 164.8]
1    (164.8, 174.5]
2    (184.2, 193.9]
3               NaN
4    (164.8, 174.5]
5    (155.1, 164.8]
6    (155.1, 164.8]
7    (155.1, 164.8]
8    (155.1, 164.8]
9    (155.1, 164.8]
Name: Height, dtype: category
Categories (5, interval[float64]): [(145.352, 155.1] < (155.1, 164.8] < (164.8, 174.5] < (174.5, 184.2] < (184.2, 193.9]]

####  1可以自定义bins,指定分割点

In [27]:
bins = [145, 155, 165, 175, 185, 195]
res2 = pd.cut(s, bins = bins)
res2.head(10)

0    (155.0, 165.0]
1    (165.0, 175.0]
2    (185.0, 195.0]
3               NaN
4    (165.0, 175.0]
5    (155.0, 165.0]
6    (155.0, 165.0]
7    (155.0, 165.0]
8    (155.0, 165.0]
9    (155.0, 165.0]
Name: Height, dtype: category
Categories (5, interval[int64]): [(145, 155] < (155, 165] < (165, 175] < (175, 185] < (185, 195]]

####  2.  另外两个常用参数为labels和retbins，分别代表了区间的名字和是否返回分割点（默认不返回）：

In [35]:
bins = [145, 165, 185, 195]
labels = ['S', 'M', 'L']
res3 = pd.cut(s, bins = bins, labels = labels, retbins = True)
res3[0]

0        S
1        M
2        L
3      NaN
4        M
      ... 
195      S
196      S
197      S
198      M
199      S
Name: Height, Length: 200, dtype: category
Categories (3, object): ['S' < 'M' < 'L']

In [36]:
res3[1]

array([145, 165, 185, 195])

从用法上来说，qcut和cut几乎没有差别，只是把bins参数变成的q参数，qcut中的q是指quantile。这里的q为整数n时，指按照n等分位数把数据分箱，还可以传入浮点列表指代相应的分位数分割点。

In [38]:
s = df.Weight
pd.qcut(s, q=4).head()

0    (33.999, 46.0]
1      (65.0, 89.0]
2      (65.0, 89.0]
3    (33.999, 46.0]
4      (65.0, 89.0]
Name: Weight, dtype: category
Categories (4, interval[float64]): [(33.999, 46.0] < (46.0, 51.0] < (51.0, 65.0] < (65.0, 89.0]]

In [39]:
pd.qcut(s, q=[0,0.2,0.8,1]).head()

0      (44.0, 69.4]
1      (69.4, 89.0]
2      (69.4, 89.0]
3    (33.999, 44.0]
4      (69.4, 89.0]
Name: Weight, dtype: category
Categories (3, interval[float64]): [(33.999, 44.0] < (44.0, 69.4] < (69.4, 89.0]]

### 2. 一般区间的构造
对于某一个具体的区间而言，其具备三个要素，即左端点、右端点和端点的开闭状态，
其中开闭状态可以指定right, left, both, neither中的一类：

In [41]:
t1 = pd.Interval(0,1, 'right') # 左开由闭
print(t1)
print(0.5 in t1) # 使用in可以判断元素是否属于区间：
print(0 in t1)
print(1 in t1)

(0, 1]
True
False
True


In [43]:
# 使用overlaps可以判断两个区间是否有交集：
t2 = pd.Interval(1,2, 'left')
t3 = pd.Interval(0.5, 2, 'both')

print(t1.overlaps(t2))
print(t1.overlaps(t3))

True
True


## 练习题

Ex1：统计未出现的类别
在第五章中介绍了crosstab函数，在默认参数下它能够对两个列的组合出现的频数进行统计汇总：

In [45]:
df = pd.DataFrame({'A':['a','b','c','a'], 'B':['cat','cat','dog','cat']})
df

,A,B
0,a,cat
1,b,cat
2,c,dog
3,a,cat


In [46]:
pd.crosstab(df.A, df.B)

B,cat,dog
A,,
a,2,0
b,1,0
c,0,1


但事实上有些列存储的是分类变量，列中并不一定包含所有的类别，此时如果想要对这些未出现的类别在crosstab结果中也进行汇总，则可以指定dropna参数为False：

In [47]:
df.B = df.B.astype('category').cat.add_categories('sheep')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   A       4 non-null      object  
 1   B       4 non-null      category
dtypes: category(1), object(1)
memory usage: 268.0+ bytes


In [48]:
pd.crosstab(df.A, df.B, dropna=False)

B,cat,dog,sheep
A,,,
a,2,0,0
b,1,0,0
c,0,1,0


问题： 请实现一个带有dropna参数的my_crosstab函数来完成上面的功能

In [60]:
def my_crosstab(s1, s2, dropna=True):
     idx1 = (s1.cat.categories if s1.dtype.name == 'category' and
                              not dropna else s1.unique())
     idx2 = (s2.cat.categories if s2.dtype.name == 'category' and
                              not dropna else s2.unique())
     res = pd.DataFrame(np.zeros((idx1.shape[0], idx2.shape[0])),
                     index=idx1, columns=idx2)
     for i, j in zip(s1, s2):
         res.at[i, j] += 1
     res = res.rename_axis(index=s1.name, columns=s2.name).astype('int')
     return res 

In [61]:
df = pd.DataFrame({'A':['a','b','c','a'],
                    'B':['cat','cat','dog','cat']})
df.B = df.B.astype('category').cat.add_categories('sheep')
my_crosstab(df.A, df.B, dropna=False)

B,cat,dog,sheep
A,,,
a,2,0,0
b,1,0,0
c,0,1,0


### Ex2：钻石数据集
现有一份关于钻石的数据集，其中carat, cut, clarity, price分别表示克拉重量、切割质量、纯净度和价格，样例如下：

In [65]:
df = pd.read_csv('diamonds.csv') 
df.head()

,carat,cut,clarity,price
0,0.23,Ideal,SI2,326
1,0.21,Premium,SI1,326
2,0.23,Good,VS1,327
3,0.29,Premium,VS2,334
4,0.31,Good,SI2,335


#### 1. 分别对df.cut在object类型和category类型下使用nunique函数，并比较它们的性能。

In [66]:
%%time
df.cut.nunique()

Wall time: 8 ms


5

In [67]:
%%time
df.cut.astype('category').nunique()

Wall time: 12 ms


5

In [68]:
%timeit -n 100 df.cut.nunique()

5.06 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [69]:
%timeit -n 100 df.cut.astype('category').nunique()

7.32 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


上面的代码不科学，因为category时，是先将obj类型转化为categories，在统计，其运行时间包含了两块
应该先将obj转化为cat,在进行时间统计

In [70]:
obj = df.cut
cat = df.cut.astype('category')

In [71]:
%timeit -n 50 obj.nunique()

4.96 ms ± 208 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


In [72]:
%timeit -n 50 cat.nunique()

1.41 ms ± 224 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


可以发现category类型的数据运行时间大幅降低

### 2. 钻石的切割质量可以分为五个等级，由次到好分别是Fair, Good, Very Good, Premium, Ideal，纯净度有八个等级，由次到好分别是I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF，请对切割质量按照由好到次的顺序排序，相同切割质量的钻石，按照纯净度进行由次到好的排序。

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   clarity  53940 non-null  object 
 3   price    53940 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 1.6+ MB


In [74]:
# cut/clarity转化为带有序列的category
cut_level = ['Fair','Good','Very Good','Premium','Ideal']
pure_level = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
df.cut = df.cut.astype('category').cat.reorder_categories(cut_level, ordered=True)
df.clarity = df.clarity.astype('category').cat.reorder_categories(pure_level, ordered=True)

In [75]:
res = df.sort_values(['cut','clarity'], ascending=[False,True])
res.head().append(res.tail())

,carat,cut,clarity,price
315,0.96,Ideal,I1,2801
535,0.96,Ideal,I1,2826
551,0.97,Ideal,I1,2830
653,1.01,Ideal,I1,2844
718,0.97,Ideal,I1,2856
41242,0.30,Fair,IF,1208
43778,0.37,Fair,IF,1440
47407,0.52,Fair,IF,1849
49683,0.52,Fair,IF,2144
50126,0.47,Fair,IF,2211


### 3. 分别采用两种不同的方法，把cut, clarity这两列按照由好到次的顺序，映射到从0到n-1的整数，其中n表示类别的个数。

In [81]:
df = pd.read_csv('diamonds.csv') 
df.cut = df.cut.astype('category')
df.cut = df.cut.cat.reorder_categories(
         df.cut.cat.categories[::-1])
df.clarity = df.clarity.astype('category')
df.clarity = df.clarity.cat.reorder_categories(
             df.clarity.cat.categories[::-1])

In [82]:
df.cut = df.cut.cat.codes # 1利用cat.codes
clarity_cat = df.clarity.cat.categories
df.clarity = df.clarity.replace(dict(zip(
             clarity_cat, np.arange(
                 len(clarity_cat))))) # 方法二：使用replace映射
df.head()

,carat,cut,clarity,price
0,0.23,2,4,326
1,0.21,1,5,326
2,0.23,3,3,327
3,0.29,1,2,334
4,0.31,3,4,335


### 4. 对每克拉的价格按照分别按照分位数q=[0.2, 0.4, 0.6, 0.8]与[1000, 3500, 5500, 18000]割点进行分箱得到五个类别Very Low, Low, Mid, High, Very High，并把按这两种分箱方法得到的category序列依次添加到原表中。

In [83]:
df['avg_qcut'] = pd.qcut(df.price/df.carat, [0,.2,.4,.6,.8,1], ['Very Low','Low','Mid','High','Very High'])
df['avg_cut'] = pd.cut(df.price/df.carat, [-np.inf,1000,3500,5500,18000,np.inf], labels=['Very Low','Low','Mid','High','Very High'])
df.head()

,carat,cut,clarity,price,avg_qcut,avg_cut
0,0.23,2,4,326,Very Low,Low
1,0.21,1,5,326,Very Low,Low
2,0.23,3,3,327,Very Low,Low
3,0.29,1,2,334,Very Low,Low
4,0.31,3,4,335,Very Low,Low


### 5. 第4问中按照整数分箱得到的序列中，是否出现了所有的类别？如果存在没有出现的类别请把该类别删除。

In [84]:
print(df.avg_cut.unique())
print("-" * 10)
print(df.avg_cut.cat.categories)

['Low', 'Mid', 'High']
Categories (3, object): ['Low' < 'Mid' < 'High']
----------
Index(['Very Low', 'Low', 'Mid', 'High', 'Very High'], dtype='object')


 - 上下不一致， 考虑用用remove_unused_categories删除未出现的类别

In [87]:
df.avg_cut = df.avg_cut.cat.remove_unused_categories()
df.avg_cut.cat.categories

Index(['Low', 'Mid', 'High'], dtype='object')

### 6.  对第4问中按照分位数分箱得到的序列，求每个样本对应所在区间的左右端点值和长度

In [89]:
interval_avg = pd.IntervalIndex(pd.qcut(df.price/df.carat, [0,.2,.4,.6,.8,1]))
interval_avg.left.to_frame(False)[0].head(3)

0    1051.162
1    1051.162
2    1051.162
Name: 0, dtype: float64

In [91]:
interval_avg.right.to_frame(False)[0].head(3)

0    2295.0
1    2295.0
2    2295.0
Name: 0, dtype: float64

In [92]:
interval_avg.length.to_frame(False)[0].head(3)

0    1243.838
1    1243.838
2    1243.838
Name: 0, dtype: float64